# Retail Insights - Key Performance Indicators (KPIs)

This notebook demonstrates basic analytics capabilities by connecting to the PostgreSQL database and calculating key retail metrics.

## Overview
- **Total Sales**: Sum of all order items (quantity * unit_price)
- **Top 10 Products**: Best-selling products by total sales
- **Sales by Country**: Revenue breakdown by geographic location
- **Order Analytics**: Order volume and customer insights


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


In [ ]:
# Database connection setup
DB_USER = os.getenv("DB_USER", "retail_user")
DB_PASSWORD = os.getenv("DB_PASSWORD", "retail_pass")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_NAME = os.getenv("DB_NAME", "retail_db")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

# Create database engine
engine = create_engine(DATABASE_URL)

print(f"Connected to database: {DB_NAME}")
print(f"Host: {DB_HOST}")


## Data Loading

Load data from all tables to understand the current dataset structure.


In [ ]:
# Load data from all tables
customers_df = pd.read_sql("SELECT * FROM customers", engine)
products_df = pd.read_sql("SELECT * FROM products", engine)
orders_df = pd.read_sql("SELECT * FROM orders", engine)
order_items_df = pd.read_sql("SELECT * FROM order_items", engine)

print("Dataset Overview:")
print(f"Customers: {len(customers_df):,} records")
print(f"Products: {len(products_df):,} records")
print(f"Orders: {len(orders_df):,} records")
print(f"Order Items: {len(order_items_df):,} records")


In [ ]:
# Display sample data from each table
print("=== CUSTOMERS SAMPLE ===")
display(customers_df.head())

print("\n=== PRODUCTS SAMPLE ===")
display(products_df.head())

print("\n=== ORDERS SAMPLE ===")
display(orders_df.head())

print("\n=== ORDER ITEMS SAMPLE ===")
display(order_items_df.head())


## KPI 1: Total Sales Revenue

Calculate the total sales revenue by summing all order items (quantity × unit_price).


In [ ]:
# Calculate total sales
total_sales_query = """
SELECT 
    SUM(quantity * unit_price) as total_sales,
    COUNT(DISTINCT order_id) as total_orders,
    COUNT(*) as total_items,
    AVG(quantity * unit_price) as avg_item_value
FROM order_items
"""

total_sales_df = pd.read_sql(total_sales_query, engine)

total_revenue = total_sales_df['total_sales'].iloc[0] if not total_sales_df.empty and total_sales_df['total_sales'].iloc[0] else 0
total_orders = total_sales_df['total_orders'].iloc[0] if not total_sales_df.empty else 0
total_items = total_sales_df['total_items'].iloc[0] if not total_sales_df.empty else 0
avg_item_value = total_sales_df['avg_item_value'].iloc[0] if not total_sales_df.empty and total_sales_df['avg_item_value'].iloc[0] else 0

print("📊 SALES OVERVIEW")
print(f"💰 Total Revenue: ${total_revenue:,.2f}")
print(f"📦 Total Orders: {total_orders:,}")
print(f"🛍️  Total Items Sold: {total_items:,}")
print(f"💵 Average Item Value: ${avg_item_value:.2f}")
if total_orders > 0:
    print(f"🧾 Average Order Value: ${total_revenue/total_orders:.2f}")
else:
    print("🧾 Average Order Value: $0.00 (No orders yet)")


## Summary

This notebook provides a foundation for retail analytics. The key insights include:

1. **Sales Overview**: Total revenue, orders, and average values
2. **Product Analysis**: Best-selling products by revenue
3. **Geographic Analysis**: Sales performance by country
4. **Customer Insights**: Top customers and spending patterns

## Next Steps

Future enhancements could include:
- Customer segmentation (RFM analysis)
- Seasonal trend analysis
- Predictive modeling for sales forecasting
- Real-time dashboards

**Note**: This analysis assumes you have data in your PostgreSQL database. If starting with an empty database, populate it using the API endpoints first.
